### Bibliothèques

In [63]:
import pandas as pd
import numpy as np
import random

## FONCTIONS

### Fonction de Création d'une Clef de Répartition

In [64]:
# Tire une clef de repartition (en %) de manière aleatoire en fonction du nombre de classe voulue et d'une valeur planché de % de classe
# !!!! UN NOMBRE DE CLASSES TROP GRAND PEUT ETRE PARADOXAL AVEC LA VALEUR PLANCHé !!!! 
# Exemple : 10 classes avec une valeur planché est de 15% -> 15%*10 = 150% donc impossible
def clef_repartition_random(nb_classe,v_planche):
    clef_repartition = [v_planche] * nb_classe
    somme_restant = 100 - sum(clef_repartition)

    for i in range(nb_classe):
        if somme_restant <= 0:
            break
        max_val = somme_restant + clef_repartition[i]
        nouvelle_valeur = random.randint(v_planche, max_val)
        somme_restant -= (nouvelle_valeur - clef_repartition[i])
        clef_repartition[i] = nouvelle_valeur
    
    clef_repartition[-1] += somme_restant
    random.shuffle(clef_repartition)
    
    return clef_repartition

### Fonction de Répartition Cumulative des Valeurs

In [65]:
# Donne la repartition des individus en fonction d'une clef de repartition et la taille de l'échantillion
# Cette repartition est sous forme d'une liste donnant les index de bornes des différentes classes
def repartition_cumulative(clef_repartition, taille_serie) :
    repartition = [round(taille_serie * x / 100) for x in clef_repartition]

    # Avec les valeurs arrondies, on peut se retrouver avec des différences de +-1, on rectifie le resultat de manière arbitraire
    while sum(repartition) > taille_serie :
        repartition[-1] -= 1
    
    while sum(repartition) < taille_serie :
        repartition[-1] += 1

    cumulative_repartition = [sum(repartition[:i+1]) for i in range(len(repartition))]
    return cumulative_repartition

### Fonction donnant l'Encadrement des Classes

In [66]:
# Donne une liste avec les valeurs des encadements en fonction de la repartition cumulative et des valeurs de la serie
def encadrements_classe(repartition_cumulative, serie) :
    valeur = serie.iloc[[i - 1 for i in repartition_cumulative]].tolist()
    valeur = [min(serie)]+ valeur 
    return valeur

### Fonction de Subsitution des Valeurs de la Serie

In [67]:
# Remplace les valeurs quantitatives d'une serie par des classes selon un encadrement donné dans une liste
# Pas parfaitement optimisé car il transforme toute la série puis revient pour complter les NaN a cause de la valeur maximum qui n'est pas prit en compte cause de la borne exclue 

def substitutions_valeurs1(serie, bins):
    # Création des labels en fonction des intervalles
    labels = [f"[{bins[i]};{bins[i+1]}[" for i in range(len(bins)-2)]
    # Dernier intervalle inclut la borne droite
    labels.append(f"[{bins[-2]};{bins[-1]}]")  
    
    # pd.cut pour classer les valeurs dans les intervalles
    classes = pd.cut(serie, bins=bins, labels=labels, right=False)
    # Complete les valeurs manquantes (maximum de la serie) par la derniere borne
    classes = classes.fillna(f"[{bins[-2]};{bins[-1]}]")
    return classes

## Test :

In [68]:
df = pd.read_csv(r'files\clean.csv')

# Garde que les variables quanti
variables_quali = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country','income']
df = df.drop(columns= variables_quali)

# Isolation de la variable age pour faire les tests
serie = df['age']

# Ordonne la série dans l'ordre croissant et reinitialiser les index
serie = serie.sort_values()
serie = serie.reset_index(drop=True)

In [69]:
# On choisi 5 classes et un minimum de 10%
clef_repartition = clef_repartition_random(5,10)
print(clef_repartition)

repartition_cumulative = repartition_cumulative(clef_repartition,len(serie))
print(repartition_cumulative)

encadrement_classe = encadrements_classe(repartition_cumulative, serie)
print(encadrement_classe)

# Serie finale tout gucci
serie_finale = substitutions_valeurs1(serie,encadrement_classe)

# passe la série en fichier csv
serie_finale.to_csv('classes_random_age_THEO.csv', index=False)

# Met la serie original au coté de la serie finale pour vérifier les transformations effectuées
# df_test_comparaison = pd.DataFrame({'age_1': serie_finale, 'age_2': serie})
# df_test_comparaison.to_csv('classes_random_age_THEO.csv', index=False)


[10, 10, 57, 10, 13]
[4522, 9044, 34821, 39343, 45222]
[17, 22, 26, 48, 54, 90]
